In [237]:
!pip install gymnasium


In [238]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

#FrozenLake-v1
###Không gian hành động


*   0: di chuyển sang trái
*   1: di chuyến xuống
*   2: di chuyển sang phải
*   3: di chuyển lên

###Không gian trạng thái
* Với bản đồ 4x4 có 16 không gian trạng thái
* Với bản đồ 8x8 có 64 không gian trạng thái
* Không gian trạng thái trong Frozen Lake được tính bằng</br>
  **current_row * nrows + current_col**

###Phần thưởng


*   Khi đến đích: +1
*   Khi sang ô băng: +0
* Khi đến hố: +0







In [239]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [240]:
env.P[0][3] # Transition model
#[starting sate][action- 3 means move up]
# the result is (probabilities, next state, reward, done)

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [241]:
env.observation_space.n #the number of possible states in the environment
#The observation is a value representing the player’s current position as current_row * nrows + current_col

16

In [242]:
env.action_space.n

4

In [243]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [244]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)
#(reward,the number of steps)

(0.0, 11)

In [245]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 12)

In [246]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Up)
SFFF
FHFH
FFFH
HFFG



(0.0, 14)

In [247]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 5)

In [248]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 30)

In [249]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [250]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)
#Với policy_0 (luôn đi sang trái) thì không có ván nào thắng

Number of successes: 0/1000
Average number of steps: nan


In [251]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)
#policy_1 có thêm khoảng 70 ván thắng trên 1000 ván chơi

Number of successes: 58/1000
Average number of steps: 12.241379310344827


In [252]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)
#policy_2 tốt hơn policy_1 có khoảng 100 ván tháng trên 1000

Number of successes: 95/1000
Average number of steps: 15.621052631578948


In [253]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)
#policy_3 tốt hơn hẳn các policy khác và thắng được gần 80% ván chơi

Number of successes: 781/1000
Average number of steps: 41.34571062740077


In [254]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [255]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)
#v_values cho các action của policy_0 đều là 0, điều này cho thấy là giá trị trạng thái khi đi sang trái đều bằng 0, tức không thể tới đích khi ở mỗi trạng thái đều đi sang trái

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [256]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)
#policy_1 ta thấy số liệu đã được phân bố, các điểm gần đích thì có giá trị trạng thái cao, trong khi đó những ô là hố thì có giá trị là 0

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [257]:
np.all(v_values_1 >= v_values_0)
#mọi giá trị trạng thái ược lượng của policy_1 >= policy_0 => pi_1>=pi_0

True

In [258]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [259]:
np.all(v_values_2 >= v_values_1)
#mọi giá trị trạng thái ược lượng của policy_2 >= policy_1 => pi_2>=pi_1

True

In [260]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [261]:
np.all(v_values_3 >= v_values_2)
#pi_3>=pi_2

True

##Value Iteration & Policy Extraction
Dùng Value Iteration để tính giá trị của mỗi trạng thái khi tuân theo một kế hoạch tối ưu </br>
Policy Extraction để tính kế hoạch từ giá trị tối ưu đó

In [262]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [263]:
import time
start_value_iteration=time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end_value_iteration=time.time()
time_value_iteration=end_value_iteration-start_value_iteration
print(time_value_iteration)

Converged at 79-th iteration.
0.07459521293640137


In [264]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [265]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    # Tất cả các chiến lược ở mỗi trạng thái đều bằng 0
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

Policy Extraction chọn một chiến lược bất kì, sau đó duyệt hết tất cả trạng thái theo từng hành động. Hành động nào của mỗi trạng thái có q_value lớn nhất thì sẽ chọn hành động đó. </br>
Từ đó tạo nên một chiến lược

In [266]:
start_policy_extraction=time.time()
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9) # Policy extraction từ value của mỗi trạng thái đã tính được từ value iteration
time_policy_extraction=end_policy_extraction-start_policy_extraction
print(time_policy_extraction)

0.0019466876983642578


In [267]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [268]:
play(env, optimal_policy, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 8)

In [269]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 783/1000
Average number of steps: 41.17496807151979


##Policy Iteration
Chọn một kế hoạch ngẫu nhiên </br>
Tính giá trị của mỗi trạng thái cho mỗi hành động của kế hoạch đó dùng **Policy Evaluation**, từ đó tính kế hoạch mới (policy improvement) dùng **Policy Extraction**.
Lặp lại cho đến khi kế hoạch không bị thay đổi.


In [270]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    # Tất cả các chiến lược ở mỗi trạng thái đều bằng 0
    #policy = np.zeros(env.observation_space.n, dtype=np.int32)
    #policy=np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
    policy=np.zeros(env.observation_space.n, dtype=np.int32)
    for i in range(max_iters):

      pre_policy=np.copy(policy)
      v_values=policy_evaluation(env, policy, max_iters=500, gamma=0.9)
      policy=policy_extraction(env,v_values,gamma=0.9)

      if np.all(np.isclose(policy, pre_policy)):
            print(f'Converged at {i}-th iteration.')
            break
    return policy

In [271]:
start_policy_iteration=time.time()
optimal_policy = policy_iteration(env, max_iters=5000, gamma=0.9)
end_policy_iteration=time.time()
time_policy_iteration=end_policy_iteration-start_policy_iteration
print(time_policy_iteration)

Converged at 0-th iteration.
Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 5-th iteration.
0.16670680046081543


In [272]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [273]:
play(env, optimal_policy, True)

  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 71)

In [274]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 772/1000
Average number of steps: 41.89248704663213


###Thống kê thời gian trong 5 lần chạy
| Lần | Value Iteration | Policy Iteration |
| --- | --- | --- |
| 1 | 0.15 | 0.1|
| 2 | 0.12 | 0.1|
| 3 | 0.14 | 0.09|
| 4 | 0.11 | 0.1|
| 5 | 0.07 | 0.16|

#FrozenLake8x8-v1

In [275]:
env = gym.make('FrozenLake8x8-v1', render_mode="ansi")

##Value Iteration

In [276]:
start_value_iteration=time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end_value_iteration=time.time()
time_value_iteration=end_value_iteration-start_value_iteration
print(time_value_iteration)

Converged at 117-th iteration.
0.7446534633636475


In [277]:
optimal_v_values

array([0.00641104, 0.00854808, 0.01230044, 0.01778942, 0.02508214,
       0.03247089, 0.03957134, 0.04297844, 0.00602405, 0.00764512,
       0.01091162, 0.01642654, 0.02605411, 0.03619409, 0.0493547 ,
       0.05730461, 0.00509024, 0.0058532 , 0.00677534, 0.        ,
       0.02557084, 0.03882139, 0.06763973, 0.08435607, 0.0042256 ,
       0.00476954, 0.00581968, 0.0078541 , 0.02036065, 0.        ,
       0.09175501, 0.12919111, 0.00318093, 0.00319659, 0.00270488,
       0.        , 0.0344439 , 0.06195145, 0.10901921, 0.20969093,
       0.00186915, 0.        , 0.        , 0.01085079, 0.03250092,
       0.06304172, 0.        , 0.36008773, 0.00118046, 0.        ,
       0.00137717, 0.00366839, 0.        , 0.11568671, 0.        ,
       0.63051379, 0.00088531, 0.00077462, 0.00092218, 0.        ,
       0.13824885, 0.32258065, 0.61443932, 0.        ])

In [278]:
start_policy_extraction=time.time()
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
end_policy_extraction=time.time()
time_policy_extraction=end_policy_extraction-start_policy_extraction
print(time_policy_extraction)

0.008600711822509766


In [279]:
optimal_policy

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 1, 3, 3, 0, 0, 2, 3,
       2, 1, 3, 3, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 2, 1, 3, 2, 0, 0, 0, 1,
       3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 0],
      dtype=int32)

In [280]:
play(env, optimal_policy, True)

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG



(1.0, 55)

In [281]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 751/1000
Average number of steps: 76.9254327563249


##Policy Iteration

In [282]:
start_policy_iteration=time.time()
optimal_policy = policy_iteration(env, max_iters=5000, gamma=0.9)
end_policy_iteration=time.time()
time_policy_iteration=end_policy_iteration-start_policy_iteration
print(time_policy_iteration)

Converged at 27-th iteration.
Converged at 27-th iteration.
Converged at 91-th iteration.
Converged at 92-th iteration.
Converged at 86-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Converged at 9-th iteration.
0.9904677867889404


In [283]:
optimal_policy

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 1, 3, 3, 0, 0, 2, 3,
       2, 1, 3, 3, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 2, 1, 3, 2, 0, 0, 0, 1,
       3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 0],
      dtype=int32)

In [284]:
play(env, optimal_policy, True)

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG



(1.0, 26)

In [285]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 734/1000
Average number of steps: 75.29019073569482


###Thống kê thời gian trong 5 lần chạy
| Lần | Value Iteration | Policy Iteration |
| --- | --- | --- |
| 1 | 0.62 | 1.63|
| 2 | 0.42 | 1|
| 3 | 0.75 | 1.01|
| 4 | 0.55 | 1.03|
| 5 | 0.74 | 0.99|

#Taxi-v3
Xe taxi khởi hành tại một ô vuông ngẫu nhiên và hành khách tại một trong những địa điểm được chỉ định
###Không gian hành động
* 0: Di chuyển xuống
* 1: Di chuyển lên
* 2: Di chuyển sang phải
* 3: Di chuyển sang trái
* 4: Đón khách
* 5: Thả khách

###Không gian trạng thái
Có 500 trạng thái riêng biệt vì có 25 vị trí taxi, 5 vị trí có thể có của hành khách (bao gồm cả trường hợp hành khách ở trong taxi) và 4 vị trí đích.</br>
*Vị trí hành khách:*
* 0: Red
* 1: Green
* 2: Yellow
* 3: Blue
* 4: In taxi

*Đích đến*
* 0: Red
* 1: Green
* 2: Yellow
* 3: Blue

*Công thức tính không gian trạng thái* </br>
((taxi_row * 5 + taxi_col) * 5 + passenger_location) * 4 + destination

###Phần thưởng
* -1 mỗi bước khi phần thưởng khác được kích hoạt.
* +20 chở hành khách.
* -10 Thực hiện hành vi “đón”, “trả” trái pháp luật.




In [286]:
env = gym.make('Taxi-v3', render_mode="ansi")

##Value Iteration

In [296]:
start_value_iteration=time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end_value_iteration=time.time()
time_value_iteration=end_value_iteration-start_value_iteration
print(time_value_iteration)

Converged at 116-th iteration.
7.655598878860474


In [288]:
optimal_v_values

array([ 89.47323891,  32.81971401,  55.26423891,  37.57755845,
         8.43222921,  32.81971401,   8.43222921,  15.28447953,
        32.81971401,  18.09386122,  55.26423891,  21.2154998 ,
        12.75594298,  18.09386122,  12.75594298,  37.57755845,
       100.52591945,  37.57755845,  62.51591945,  42.86394891,
        79.52591945,  28.53774704,  48.73781945,  32.81971401,
        10.48035311,  37.57755845,  10.48035311,  18.09386122,
        28.53774704,  15.28447953,  48.73781945,  18.09386122,
        15.28447953,  21.2154998 ,  15.28447953,  42.86394891,
        89.47323891,  42.86394891,  55.26423891,  48.73781945,
        42.86394891,  12.75594298,  24.68388374,  15.28447953,
        24.68388374,  70.57323891,  24.68388374,  37.57755845,
        24.68388374,  12.75594298,  42.86394891,  15.28447953,
        18.09386122,  24.68388374,  18.09386122,  48.73781945,
        48.73781945,  79.52591945,  48.73781945,  55.26423891,
        37.57755845,  10.48035311,  21.2154998 ,  12.75

In [297]:
start_policy_extraction=time.time()
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
end_policy_extraction=time.time()
time_policy_extraction=end_policy_extraction-start_policy_extraction
print(time_policy_extraction)

0.0588831901550293


In [298]:
optimal_policy

array([4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 3, 1, 3, 2, 3, 3, 3, 3,
       1, 1, 1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 3, 3, 3, 3, 1, 1,
       1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 1,

In [299]:
optimal_policy_1=optimal_policy

In [290]:
play(env, optimal_policy, True)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



(5, 16)

In [300]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 1000/1000
Average number of steps: 13.128


##Policy Iteration

In [301]:
start_policy_iteration=time.time()
optimal_policy = policy_iteration(env, max_iters=5000, gamma=0.9)
end_policy_iteration=time.time()
time_policy_iteration=end_policy_iteration-start_policy_iteration
print(time_policy_iteration)

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


Converged at 88-th iteration.
Converged at 88-th iteration.
Converged at 97-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 16-th iteration.
18.980047464370728


In [302]:
optimal_policy

array([4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 3, 1, 3, 2, 3, 3, 3, 3,
       1, 1, 1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 3, 3, 3, 3, 1, 1,
       1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 1,

In [294]:
play(env, optimal_policy, True)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



(11, 10)

In [303]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 1000/1000
Average number of steps: 13.039


In [307]:
# Kiểm tra policy của 2 thuật toán có giống nhau không
if (np.array_equal(optimal_policy_1, optimal_policy)):
  print("Equal")
else: print("Not Equal")

Equal


###Thống kê thời gian trong 5 lần chạy
| Lần | Value Iteration | Policy Iteration |
| --- | --- | --- |
| 1 | 5.67 | 15.27|
| 2 | 6.22 | 13.12|
| 3 | 6.2 | 15.29|
| 4 | 6.2 | 15.36|
| 5 | 6.45 | 18.05 |

##Nhận xét
| Value Iteration | Policy Iteration
| ---------- | ---------- |
| Cập nhật cả giá trị và ngầm định cập nhật cả kế hoạch  | Chúng ta cập nhật giá trị của hành động theo kế hoạch <br> (nhanh vì chỉ xét 1 hành động) |
| Chúng ta không tìm kế hoạch, nhưng mỗi lấy giá trị lớn nhất <br> trong các hành động là ngầm tìm một kế hoạch | Sau đó tìm một kế hoạch mới dựa trên giá trị đã tính <br> (Giai đoạn này chậm như Value Iteration)

*   **Về tốc độ** : Value Iteration cho ra kết quả nhanh hơn bởi vì nó yêu cầu ít vòng lặp hơn Policy Iteration. Value Iteration kết hợp tính toán và cải thiện trong mỗi bước đi, trong khi đó Policy Iteration tính toán và cải thiệt cả một kế hoạch trong mỗi vòng lặp.
*  **Hiệu suất tính toán** : Policy Iteration yêu cầu tính toán nhiều hơn vì có nhiều vòng lặp.
* **Chất lượng kế hoạch** : Policy Iteration đảm bảo có một kế hoạch tối ưu hơn Value Iteration.Vì Policy Iteration tìm một chính sách tối ưu, trong khi đó Value Iteration tìm một chính sách tối ưu từ giá trị trạng thái.
* Policy Iteration được ưu tiên khi cần tìm một giải pháp yêu cầu sự chính xác, và không bị giới hạn bộ nhớ hay thời gian. Value Iteration phù hợp với môi trường không yêu cầu độ chính xác cao và thời gian cho kết quả nhanh.






##Thuật toán tốt nhất
Trong Frozen Lake các policy khi sử dụng 2 thuật toán đều giống nhau, trung bình thời gian để tìm ra giải pháp tối ưu Value Iteration chậm hơn Policy Iteration trong Frozen Lake 4x4 (không đáng kế), trong khi Value Iteration nhanh hơn Policy Iteration trong Frozen Lake 8x8. Có thể suy ra là Policy Iteration cho kết quả nhanh hơn với những bản đồ nhỏ <br>
Trong Taxi thì kế hoạch của 2 thuật toán cho ra đều như nhau và Value Iteration cho ra kết quả nhanh hơn Policy Iteration. <br>
Frozen Lake có yêu tố ngẫu nhiên, xác suất của mỗi hành động là 1/3. Trong đó, Taxi không có yếu tố ngẫu nhiên nên các kế hoạch của hai thuật toán, khi chạy nhiều lần đều đạt đích.
Vậy nên nếu xét chiến thuật nào tốt hơn thì ta ưu tiên về thời gian chạy, thì **Value Iteration là tốt hơn** Policy Iteration cho cả 3 toy game.
